# Water quelity prediction version 2 and new feature
## 1. Read data

In [9]:
import pandas as pd
import numpy as np

In [10]:
data = pd.read_csv('train.csv')

In [11]:
data.head(10)

,station_id,Aluminium_2019,Aluminium_2020,Ammonium_2019,Ammonium_2020,Boron_2019,Boron_2020,Chloride_2019,Chloride_2020,Coli-like-bacteria-Colilert_2019,...,Taste-ball-units_2020,Taste-dilution-degree_2019,Taste-dilution-degree_2020,Turbidity-NTU_2019,Turbidity-NTU_2020,pH _2019,pH _2020,compliance_2019,compliance_2020,compliance_2021
0,487,NaN,NaN,0.05,0.05,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,1.60,0.2,7.80,7.60,0,0,0
1,1555,NaN,NaN,0.05,0.05,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.00,1.0,7.30,7.10,0,0,0
2,205,NaN,10.0,0.05,0.24,NaN,0.100,NaN,NaN,NaN,...,NaN,1.0,1.0,1.00,1.0,7.80,7.70,0,0,0
3,1228,NaN,NaN,0.09,0.05,0.641,0.932,NaN,NaN,NaN,...,1.0,NaN,NaN,0.50,0.5,7.44,7.58,0,0,0
4,470,NaN,NaN,0.06,0.05,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.20,44.0,7.70,8.00,0,1,0
5,614,NaN,NaN,0.12,0.13,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,0.50,0.5,8.64,8.45,0,0,1
6,1420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,1.0,2.0,1.00,1.0,7.50,8.00,0,0,0
7,188,NaN,NaN,0.23,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.50,0.5,7.45,7.40,1,1,0
8,1286,NaN,50.0,0.20,0.23,NaN,0.091,NaN,6.5,NaN,...,NaN,NaN,NaN,0.72,0.5,7.50,7.70,1,0,1
9,736,NaN,NaN,0.05,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,0.50,0.5,7.53,7.26,0,0,0


In [16]:
data.corr()['compliance_2021'].sort_values(ascending=False)

compliance_2021                     1.000000
Smell-ball-units_2020               0.257513
Smell-ball-units_2019               0.246605
compliance_2020                     0.235502
Colony-count-at-22-C_2020           0.220534
Taste-ball-units_2020               0.216994
Taste-ball-units_2019               0.212303
compliance_2019                     0.190970
Ammonium_2020                       0.187687
Sodium_2019                         0.176823
Coli-like-bacteria-Colilert_2020    0.166834
Taste-dilution-degree_2020          0.155139
Oxidability_2019                    0.150060
Odour-dilution-level_2020           0.145637
Manganese_2019                      0.138797
Chloride_2019                       0.126462
Colony-count-at-22-C_2019           0.119071
Color-Pt/Co-scale_2019              0.112381
Manganese_2020                      0.104825
Ammonium_2019                       0.100698
station_id                          0.098545
Nitrate_2019                        0.092281
Chloride_2

In [17]:
# new features based on two prev results looks logical 0.133063 corr
data['prev_two_probas'] = np.where((data['compliance_2019'] & data['compliance_2020'])==1,1,0)
data.corr()['compliance_2021'].sort_values(ascending=False)

compliance_2021                     1.000000
Smell-ball-units_2020               0.257513
Smell-ball-units_2019               0.246605
compliance_2020                     0.235502
Colony-count-at-22-C_2020           0.220534
Taste-ball-units_2020               0.216994
Taste-ball-units_2019               0.212303
compliance_2019                     0.190970
Ammonium_2020                       0.187687
Sodium_2019                         0.176823
Coli-like-bacteria-Colilert_2020    0.166834
Taste-dilution-degree_2020          0.155139
Oxidability_2019                    0.150060
Odour-dilution-level_2020           0.145637
Manganese_2019                      0.138797
prev_two_probas                     0.133063
Chloride_2019                       0.126462
Colony-count-at-22-C_2019           0.119071
Color-Pt/Co-scale_2019              0.112381
Manganese_2020                      0.104825
Ammonium_2019                       0.100698
station_id                          0.098545
Nitrate_20

In [18]:
from sklearn.impute import SimpleImputer
X = data.drop(['compliance_2021'],axis=1)
y = data['compliance_2021']
imputer = SimpleImputer(strategy = 'median')
imputer.fit(X)

SimpleImputer(strategy='median')

In [19]:
from sklearn.utils import resample
print(data[data['compliance_2021']==0]['compliance_2021'].value_counts())
df_majority = data[data['compliance_2021']==1]
df_minority = data[data['compliance_2021']==0]

df_majority_downsampled = resample(df_majority, 
                                 replace=True,    # sample without replacement
                                 n_samples=374,     # to match minority class
                                 random_state=123) # reproducible results


# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled.sort_index(inplace=True)

y = df_downsampled['compliance_2021']
X = df_downsampled.drop('compliance_2021', axis=1)
print(y.value_counts())

0    374
Name: compliance_2021, dtype: int64
0    374
1    374
Name: compliance_2021, dtype: int64


In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 7)

In [21]:
X_train = pd.DataFrame(imputer.transform(X_train))
X_test = pd.DataFrame(imputer.transform(X_test))

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

rfc = RandomForestClassifier(random_state=78)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

confusion_matrix_result = confusion_matrix(y_test.values, y_pred)
print("Confusion matrix:\n%s" % confusion_matrix_result)
accuracy_score(rfc.predict(X_test), y_test)

print(f'Accuracy of model on the test data {accuracy_score(rfc.predict(X_test), y_test)}')

Confusion matrix:
[[62  3]
 [ 0 85]]
Accuracy of model on the test data 0.98


In [23]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

cv_dict = cross_val_score(RandomForestClassifier(random_state = 78), X_train, y_train, cv=10)



In [24]:
cv_dict.mean()

0.9648305084745761

In [25]:
test_data = pd.read_csv('test.csv')
test_data.head()
test_data['prev_two_probas'] = np.where((test_data['compliance_2019'] & test_data['compliance_2020'])==1,1,0)
test_data.head()

,station_id,Aluminium_2019,Aluminium_2020,Ammonium_2019,Ammonium_2020,Boron_2019,Boron_2020,Chloride_2019,Chloride_2020,Coli-like-bacteria-Colilert_2019,...,Taste-ball-units_2020,Taste-dilution-degree_2019,Taste-dilution-degree_2020,Turbidity-NTU_2019,Turbidity-NTU_2020,pH _2019,pH _2020,compliance_2019,compliance_2020,prev_two_probas
0,163,5.0,5.0,0.08,0.08,0.071,0.062,130.0,102.0,NaN,...,NaN,1.0,1.0,1.18,1.90,8.17,8.12,0,0,0
1,167,NaN,NaN,0.08,0.08,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,3.0,3.90,1.54,7.83,7.81,1,0,0
2,171,NaN,NaN,NaN,NaN,NaN,NaN,112.0,90.0,NaN,...,NaN,2.0,1.0,1.40,1.50,7.50,7.60,0,1,0
3,174,5.0,5.0,0.05,0.09,0.072,0.075,248.0,243.0,0.0,...,NaN,2.0,8.0,1.00,1.00,7.70,7.80,0,0,0
4,178,NaN,NaN,0.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.00,1.00,7.50,7.40,0,0,0


In [26]:
imputer.fit(test_data)

test_data_mod = pd.DataFrame(imputer.transform(test_data))
y_pred = rfc.predict(test_data_mod)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [27]:
df = pd.DataFrame()
df['station_id'] = test_data.station_id
df['compliance_2021'] = y_pred
df.to_csv('water.csv', index=False)

df.value_counts()

station_id  compliance_2021
163         0                  1
1072        0                  1
985         0                  1
986         0                  1
988         0                  1
                              ..
549         0                  1
551         0                  1
559         0                  1
583         0                  1
2307        0                  1
Length: 189, dtype: int64

90.4 % kaggle nice 